# Boston Housing Data Analysis

# Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(precision=3, suppress=True)

# Importing the dataset

In [2]:
from helper import boston_dataframe
boston_data = boston_dataframe(description = True)

In [3]:
ds = boston_data[0]
ds_desc = boston_data[1]

In [4]:
ds.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
ds.shape

(506, 14)

In [6]:
X = ds.drop('MEDV', axis = 1)
y = ds.MEDV

# Taking care of missing data

In [7]:
# We observe no missing data
ds.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

# Encoding categorical data

In [8]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
MEDV       506 non-null float64
dtypes: float64(14)
memory usage: 55.5 KB


# Feature scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

In [10]:
X_scaled = sc_X.fit_transform(X)

# Confirm Scaling

In [11]:
a = np.array([[1,2,3],
              [4,5,6]])

In [12]:
a

array([[1, 2, 3],
       [4, 5, 6]])

In [13]:
# Mean along the columns
a.mean(axis = 0)

array([2.5, 3.5, 4.5])

In [14]:
# Mean along the rows
a.mean(axis = 1)

array([2., 5.])

In [15]:
X2 = np.array(X_scaled)
manual_transform = (X2-X2.mean(axis = 0) / X2.std(axis = 0))
np.allclose(manual_transform, X2)

True

# Comparing the coefficients with and without scaling

In [16]:
# Without scaling

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X, y)
print(pd.DataFrame(lr.coef_))

            0
0   -0.108011
1    0.046420
2    0.020559
3    2.686734
4  -17.766611
5    3.809865
6    0.000692
7   -1.475567
8    0.306049
9   -0.012335
10  -0.952747
11   0.009312
12  -0.524758


In [17]:
# With scaling

from sklearn.linear_model import LinearRegression
lr2 = LinearRegression()
lr2.fit(X_scaled, y)
print(pd.DataFrame(lr2.coef_))

           0
0  -0.928146
1   1.081569
2   0.140900
3   0.681740
4  -2.056718
5   2.674230
6   0.019466
7  -3.104044
8   2.662218
9  -2.076782
10 -2.060607
11  0.849268
12 -3.743627


From the below, we observe the most "impactful" features.

We will use the below code to display our desired outcome:

`dict(zip(df.columns.values, model.coef_))`

In [18]:
pd.DataFrame(zip(X.columns, lr2.coef_)).sort_values(by=1)

,0,1
12,LSTAT,-3.743627
7,DIS,-3.104044
9,TAX,-2.076782
10,PTRATIO,-2.060607
4,NOX,-2.056718
0,CRIM,-0.928146
6,AGE,0.019466
2,INDUS,0.140900
3,CHAS,0.681740
11,B,0.849268


By observing the strength of the standardised coefficients LSTAT, DIS, RAD and RM, we state that they are the 'most impactful'.

## Lasso with and without scaling

In [19]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures # polynomial + regularisation

In [20]:
# Fitting the polynomial terms to the dataset

pf = PolynomialFeatures(degree = 2, include_bias = False)
X_pf = pf.fit_transform(X)

In [21]:
# Feature Scaling

sc = StandardScaler()
X_pf_sc = sc.fit_transform(X_pf)

In [22]:
# Fitting Lasso to the dataset
# Note: Lasso has a default alpha value of 1

lasso = Lasso()
lasso.fit(X_pf_sc, y)
lasso.coef_

array([-0.   ,  0.   , -0.   ,  0.   , -0.   ,  0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.991,  0.   , -0.   , -0.   ,  0.   , -0.   ,
        0.068, -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.   ,  0.   ,
       -0.   , -0.   , -0.   , -0.05 , -0.   , -0.   , -0.   , -0.   ,
       -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   ,  0.   , -0.   ,  3.3  , -0.   , -0.   , -0.   ,
       -0.   , -0.   ,  0.42 , -3.498, -0.   , -0.   , -0.   , -0.   ,
       -0.   ,  0.   , -0.   , -0.   , -0.   , -0.146, -0.   , -0.   ,
       -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.   ,  0.   , -0.   ,  0.   , -0.   , -0.   ])

In [23]:
# Lasso for alpha = 0.1

lasso1 = Lasso(alpha = 0.1)
lasso1.fit(X_pf_sc, y)
print('Sum of coefficients:', abs(lasso1.coef_).sum())
print('Number of coefficients not equal to 0:', (lasso1.coef_!=0).sum())

Sum of coefficients: 26.138682362877972
Number of coefficients not equal to 0: 23


In [24]:
# Lasso for alpha = 1

lasso2 = Lasso(alpha = 1)
lasso2.fit(X_pf_sc, y)
print('Sum of coefficients:',abs(lasso2.coef_).sum())
print('Number of coefficients not equal to 0:',(lasso2.coef_!=0).sum())

Sum of coefficients: 8.472405044553074
Number of coefficients not equal to 0: 7


With more regularisation (higher alpha) we will expect the penalty for higher weights to be greater and thus the coefficients to be pushed down. Thus a higher alpha means lower magnitude with more coefficients pushed down to 0.

## Calculate the $R^{2}$ of each model without train/test split.

In [25]:
from sklearn.metrics import r2_score

In [26]:
print(r2_score(y, lasso1.predict(X_pf_sc)))

0.8366045990855128


In [27]:
print(r2_score(y, lasso2.predict(X_pf_sc)))

0.7207000417838496


If we evaluate our model on the same dataset, we may be overfitting the dataset. We will therefore analyse the performance on testing data, i.e. splitting the dataset into the training set and test set.

# Splitting the dataset into the training set and test set

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size = 0.3, random_state = 72018)

In [29]:
# Feature Scaling

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [30]:
lasso1.fit(X_train, y_train)
y_pred = lasso1.predict(X_test)
print(r2_score(y_test, y_pred))

0.7995486651220819


In [31]:
lasso2.fit(X_train, y_train)
y_pred = lasso2.predict(X_test)
print(r2_score(y_test, y_pred))

0.677935367178621


**Next steps:**

- Let's do the same thing with Lasso of:
    - `alpha` of 0.001
    - Increase `max_iter` to 100000 to ensure convergence.

- Calculate the  $R^{2}$  of the model.
- Do the same procedure as before, but with Linear Regression.
- Calculate the  $R^{2}$  of this model.
- Compare the sums of the absolute values of the coefficients for both models, as well as the number of coefficients that are zero. 

**Analysis:** Based on these measures, which model is a "simpler" description of the relationship between the features and the target?

**Note:** From the below we observe that LASSO did a better job in explaining the additional variation.

In [32]:
# Decreasing regularisation and ensuring convergence

lasso3 = Lasso(alpha = 0.001, max_iter = 100000)
lasso3.fit(X_train, y_train)
y_pred = lasso3.predict(X_test)
print('Sum of coefficients:',abs(lasso3.coef_).sum())
print('Number of coefficients not equal to 0:',(lasso3.coef_!=0).sum())
print('The R2 score is:', r2_score(y_test, y_pred))

Sum of coefficients: 436.26164263064214
Number of coefficients not equal to 0: 89
The R2 score is: 0.8848141421367259


In [33]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('Sum of coefficients:',abs(lr.coef_).sum())
print('Number of coefficients not equal to 0:',(lr.coef_!=0).sum())
print('The R2 score is:', r2_score(y_test, y_pred))

Sum of coefficients: 1185.285825445829
Number of coefficients not equal to 0: 104
The R2 score is: 0.8687609133106025


# L1 vs. L2 Regularization

As previously mentioned, `Lasso` and `Ridge` regression have the same syntax in SciKit Learn.

Now we're going to compare the results from Ridge vs. Lasso regression.

In [34]:
from sklearn.linear_model import Ridge

# Decreasing regularisation and ensuring convergence

rr = Ridge(alpha = 0.001, max_iter = 100000)
rr.fit(X_train, y_train)
y_pred = rr.predict(X_test)
rr.coef_

array([  8.727,  10.354, -24.683,   5.299,  -3.13 ,  14.781,  22.301,
       -23.483,  27.356,  -1.512,  16.949,  22.531,  11.444,   1.055,
         0.437,  14.259,   1.83 ,  -8.737,   4.726,  -3.748,  -3.557,
       -16.521,  -6.918,   6.064,  -1.363,   4.604,  -1.303,  -0.057,
        -0.301, -12.828,   2.016,   1.084,  -0.655,  -1.092,   4.458,
        -4.247,   4.156,  -1.282,   8.71 ,  -0.285,  -8.595,  11.595,
         6.544,   1.271,   1.827,   1.007,   1.554,   5.173,  -4.685,
         5.299,  -3.199,  -8.715,   0.983,   1.119,   0.284,  -1.683,
        -2.884,   2.985,  -0.756,  12.297,   0.931,  -7.594,  18.292,
       -22.124,  35.725, -21.894,  -7.789,   1.774,   4.101, -12.263,
        -3.741,  -5.525, -16.111,  -5.706,  -4.666,  -9.884,   0.989,
        -0.092,  17.145, -14.214,  -2.955,  -2.785,  -5.404,  11.825,
         0.085,  -4.352,  -5.616,  -2.843,   0.826, -29.129,  49.636,
       -21.571,  -1.263,  -8.811, -15.822,  11.975,  -1.023,   2.911,
        -0.656,  -4.

In [35]:
print('Sum of coefficients:',abs(rr.coef_).sum())
print('Number of coefficients not equal to 0:',(rr.coef_!=0).sum())
print('The R2 score is:', r2_score(y_test, y_pred))

Sum of coefficients: 795.6521694306358
Number of coefficients not equal to 0: 104
The R2 score is: 0.8701706652191431


In [36]:
# We observe that LASSO with alpha 0.001 has more zeros than Ridge with alpha 0.001
# This is becuase, by default, LASSO will always zero more values than Ridge will.

lasso3.coef_

array([  0.   ,   0.   , -16.945,   2.562,   0.   ,  13.392,   9.907,
       -19.96 ,   9.039,   0.   ,   6.138,  17.628,  11.446,   1.21 ,
         0.226,  11.578,   2.197,  -7.397,   4.117,  -1.589,  -2.157,
        -9.837,   0.   ,   0.   ,  -1.089,   3.793,   0.389,   0.2  ,
        -0.297,  -3.352,   0.396,   0.719,  -0.756,  -0.753,   2.388,
        -0.845,   2.524,  -0.973,   3.882,  -0.983,   6.609,   6.138,
         4.201,   0.858,   1.96 ,   2.661,  -4.148,   2.937,  -4.556,
         2.112,  -1.983,  -7.439,   1.607,   1.668,  -1.262,  -0.   ,
        -0.   ,   2.622,  -0.861,   3.353,  -0.   ,  -2.451,   9.171,
        -6.917,   0.   ,  -5.618,  -5.716,   0.521,   3.354,  -7.528,
        -0.   ,  -6.646, -10.212,  -6.346,  -2.95 ,  -9.559,   0.327,
         0.56 ,  11.006,  -6.666,  -1.158,  -2.25 ,  -3.53 ,   9.035,
        -0.   ,  -2.316,  -2.   ,  -1.124,   0.804, -20.861,  27.166,
         0.   ,  -1.51 ,  -6.745,  -5.098,  15.455,   0.   ,   0.   ,
         0.629,  -3.

In [37]:
print('Sum of coefficients:',abs(lasso3.coef_).sum())
print('Number of coefficients not equal to 0:',(lasso3.coef_!=0).sum())
print('The R2 score is:', 0.8848141421367259) # LASSO has done a better generalising

Sum of coefficients: 436.26164263064214
Number of coefficients not equal to 0: 89
The R2 score is: 0.8848141421367259


# Conclusion

Ridge does not make any coefficients 0. In addition, on this particular dataset, Lasso provides stronger overall regularization than Ridge for this value of alpha (not necessarily true in general).